In [10]:
import pandas as pd
import numpy as np
import random
import string

def randomString(stringLength=10):
    """Generate a random string of fixed length """
    letters = string.ascii_lowercase
    rand_int = random.randint(0,2)
#     print(rand_int)
    string_random = ''.join(random.choice(letters) for i in range(stringLength))
#     for i in range(rand_int+1):
    
#         string_random = string_random +" " + ''.join(random.choice(letters) for i in range(stringLength))
    
    return "Destination_"+string_random

print(randomString())

Destination_jefcozzdnc


In [46]:
print(randomString())

Destination_rblsiwiytu


In [47]:
data = open("dummyData_augmented.txt").readlines()

In [48]:
import spacy

nlp = spacy.load("en_core_web_sm")


In [49]:
array = []
for textindex,text in enumerate(data):
    doc = nlp(text.strip())
    for token in doc:
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        #        token.shape_, token.is_alpha, token.is_stop)
        text = token.text
        if token.text == "XYYZ":
            tag = "Destination"
            text = randomString()
        else:
            tag = ""
        
        array.append(dict(Sentence=textindex,Word=text,POS=token.pos_,Tag=tag))

In [50]:
df = pd.DataFrame(array)  

In [51]:
df.tail(10)

,POS,Sentence,Tag,Word
1021,NOUN,169,,grocery
1022,NOUN,169,,store
1023,PRON,170,,What
1024,AUX,170,,do
1025,PRON,170,,I
1026,VERB,170,,look
1027,ADP,170,,for
1028,PUNCT,170,,?
1029,ADJ,170,,nearest
1030,NOUN,170,,starbucks


In [52]:
words = list(set(df["Word"].values))
words.append("ENDPAD")
words.append("Unknown")
n_words = len(words); n_words

192

In [53]:
tags = list(set(df["Tag"].values))
n_tags = len(tags); n_tags

pos = list(set(df["POS"].values))
pos_tags = len(pos)

In [54]:
agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                   s["POS"].values.tolist(),
                                                   s["Tag"].values.tolist())]
grouped = df.groupby("Sentence").apply(agg_func)

In [55]:
sentences = [s for s in grouped]

In [56]:
print(sentences[0])

[('directions', 'NOUN', ''), ('to', 'ADP', ''), ('Destination_otajnpqefx', 'PROPN', 'Destination')]


In [57]:
def yield_sentence():
    for group in grouped:
        yield group
    yield None

In [58]:
for group in yield_sentence():
    print(group)
    print("-------")

[('directions', 'NOUN', ''), ('to', 'ADP', ''), ('Destination_otajnpqefx', 'PROPN', 'Destination')]
-------
[('directions', 'NOUN', ''), ('to', 'ADP', ''), ('Destination_janjwafhfp', 'PROPN', 'Destination'), ('home', 'ADV', '')]
-------
[('directions', 'NOUN', ''), ('to', 'ADP', ''), ('Destination_lixwjfkfnc', 'PROPN', 'Destination'), ('work', 'NOUN', '')]
-------
[('directions', 'NOUN', ''), ('to', 'ADP', ''), ('the', 'DET', ''), ('closest', 'ADJ', ''), ('Destination_xsbwunyncm', 'PROPN', 'Destination')]
-------
[('directions', 'NOUN', ''), ('to', 'ADP', ''), ('the', 'DET', ''), ('nearest', 'ADJ', ''), ('Destination_tbtylkmxuu', 'PROPN', 'Destination')]
-------
[('directions', 'NOUN', ''), ('to', 'PART', ''), ('go', 'VERB', ''), ('home', 'ADV', '')]
-------
[('directions', 'NOUN', ''), ('to', 'PART', ''), ('go', 'VERB', ''), ('to', 'ADP', ''), ('Destination_fgpafiddoy', 'PROPN', 'Destination')]
-------
[('directions', 'NOUN', ''), ('to', 'PART', ''), ('go', 'VERB', ''), ('to', 'ADP', 

In [59]:
max_len = 75
word2idx = {w: i + 1 for i, w in enumerate(words) if "Destination_" not in w}
tag2idx = {t: i for i, t in enumerate(tags)}


In [60]:
word2idx["directions"]

55

In [61]:
tag2idx[""]

0

In [62]:
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx.get(w[0],word2idx["Unknown"]) for w in s] for s in sentences]

In [63]:
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words-1)


In [64]:
y = [[tag2idx[w[2]] for w in s] for s in sentences]

In [65]:
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx[""])


In [66]:
from keras.utils import to_categorical

In [67]:
y = [to_categorical(i, num_classes=n_tags) for i in y]

In [68]:
from sklearn.model_selection import train_test_split

In [69]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)



In [1]:
import pickle
import pandas as pd
import numpy as np
import random
import string
pickle_in = open("training_and_testing_data.pickle","rb")

data_dict = pickle.load(pickle_in)


In [2]:

X_tr = data_dict["X_tr"]
X_te = data_dict["X_te"]
y_tr = data_dict["y_tr"]
y_te = data_dict["y_te"]
max_len = 75
n_words = data_dict["n_words"]
n_tags = data_dict["n_tags"]
tag2idx = data_dict["tag2idx"]
pos2idx = data_dict["pos2idx"]
word2idx = data_dict["word2idx"]


In [3]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
import keras

Using TensorFlow backend.


In [4]:
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words + 1, output_dim=20,
                  input_length=max_len)(input)  # 20-dim embedding
model = Bidirectional(LSTM(units=50, return_sequences=True,
                           recurrent_dropout=0.1))(model)  # variational biLSTM
model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer
crf = CRF(n_tags)  # CRF layer
out = crf(model)  # output

In [5]:
model = Model(input, out)

In [6]:
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

/home/premraj/anaconda3/lib/python3.7/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/home/premraj/anaconda3/lib/python3.7/site-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


In [7]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 75)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 75, 20)            1860      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 75, 100)           28400     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 75, 50)            5050      
_________________________________________________________________
crf_1 (CRF)                  (None, 75, 2)             110       
Total params: 35,420
Trainable params: 35,420
Non-trainable params: 0
_________________________________________________________________


In [8]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=100,
                    validation_split=0.1, verbose=1)

/home/premraj/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 144 samples, validate on 17 samples
Epoch 1/100
144/144 [==============================] - 3s 18ms/step - loss: 0.2007 - crf_viterbi_accuracy: 0.9798 - val_loss: 0.1304 - val_crf_viterbi_accuracy: 0.9812
Epoch 2/100
144/144 [==============================] - 0s 3ms/step - loss: 0.1013 - crf_viterbi_accuracy: 0.9796 - val_loss: 0.0787 - val_crf_viterbi_accuracy: 0.9812
Epoch 3/100
144/144 [==============================] - 0s 3ms/step - loss: 0.0802 - crf_viterbi_accuracy: 0.9803 - val_loss: 0.0740 - val_crf_viterbi_accuracy: 0.9812
Epoch 4/100
144/144 [==============================] - 0s 3ms/step - loss: 0.0758 - crf_viterbi_accuracy: 0.9793 - val_loss: 0.0715 - val_crf_viterbi_accuracy: 0.9812
Epoch 5/100
144/144 [==============================] - 0s 3ms/step - loss: 0.0730 - crf_viterbi_accuracy: 0.9793 - val_loss: 0.0696 - val_crf_viterbi_accuracy: 0.9812
Epoch 6/100
144/144 [==============================] - 0s 3ms/step - loss: 0.0709 - crf_viterbi_accuracy: 0.9798 - val_

144/144 [==============================] - 0s 3ms/step - loss: 0.0116 - crf_viterbi_accuracy: 0.9997 - val_loss: 0.0247 - val_crf_viterbi_accuracy: 0.9969
Epoch 50/100
144/144 [==============================] - 0s 3ms/step - loss: 0.0117 - crf_viterbi_accuracy: 0.9997 - val_loss: 0.0271 - val_crf_viterbi_accuracy: 0.9969
Epoch 51/100
144/144 [==============================] - 0s 3ms/step - loss: 0.0114 - crf_viterbi_accuracy: 0.9998 - val_loss: 0.0292 - val_crf_viterbi_accuracy: 0.9969
Epoch 52/100
144/144 [==============================] - 0s 3ms/step - loss: 0.0112 - crf_viterbi_accuracy: 0.9996 - val_loss: 0.0283 - val_crf_viterbi_accuracy: 0.9969
Epoch 53/100
144/144 [==============================] - 0s 3ms/step - loss: 0.0112 - crf_viterbi_accuracy: 0.9995 - val_loss: 0.0273 - val_crf_viterbi_accuracy: 0.9969
Epoch 54/100
144/144 [==============================] - 0s 3ms/step - loss: 0.0109 - crf_viterbi_accuracy: 0.9997 - val_loss: 0.0246 - val_crf_viterbi_accuracy: 0.9969
Epoch

Epoch 98/100
144/144 [==============================] - 0s 3ms/step - loss: 0.0075 - crf_viterbi_accuracy: 1.0000 - val_loss: 0.0344 - val_crf_viterbi_accuracy: 0.9969
Epoch 99/100
144/144 [==============================] - 0s 3ms/step - loss: 0.0074 - crf_viterbi_accuracy: 1.0000 - val_loss: 0.0353 - val_crf_viterbi_accuracy: 0.9969
Epoch 100/100
144/144 [==============================] - 0s 3ms/step - loss: 0.0072 - crf_viterbi_accuracy: 1.0000 - val_loss: 0.0361 - val_crf_viterbi_accuracy: 0.9969


In [9]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report


In [10]:
test_pred = model.predict(X_te, verbose=1)

18/18 [==============================] - 0s 16ms/step


In [11]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", ""))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)

In [12]:
def getRecallPrecision(test_labels,pred_labels,tag):
    flat_test_labels = [item for sublist in test_labels for item in sublist]
    flat_pred_labels = [item for sublist in pred_labels for item in sublist]
    print(flat_test_labels)
    print(flat_pred_labels)
    test_indices = set([i for i, x in enumerate(flat_test_labels) if x == tag])
    pred_indices = set([i for i, x in enumerate(flat_pred_labels) if x == tag])
    recall = len(test_indices.intersection(pred_indices))/len(test_indices)
    precision = len(test_indices.intersection(pred_indices))/len(pred_indices)
    f_score = 2* precision* recall/(precision + recall)
    return recall,precision,f_score

    

In [13]:
print(len(test_labels))
print([len(t) for t in test_labels])
print("-----")
print(len(pred_labels))
print([len(t) for t in pred_labels])
print(getRecallPrecision(test_labels,pred_labels,"Destination"))

18
[75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75]
-----
18
[75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75]
['', '', '', 'Destination', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'Destination', 'Destination', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'Destination', 'Destination', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''

In [14]:
import spacy
import pandas as pd
import numpy as np
import random
import string
import spacy
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import pickle
nlp = spacy.load("en_core_web_sm")

In [20]:
sentence = "Where do you want to go? 56 Torstrasse"
entities_words = ["joy","home","my","post","prem","Berlin","Festival"]
array = []

doc = nlp(sentence)
for token in doc:
    text = token.text
    print(text)
    if token.text in entities_words:
        tag = "Destination"
    else:
        tag = ""

    array.append(tuple([token.text,token.pos_,tag]))
print(array)
X = [[word2idx.get(w[0],word2idx["Unknown"]) for w in s] for s in [array]]
print(X)
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words-1)
y = [[tag2idx[w[2]] for w in s] for s in [array]]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx[""])
y = [to_categorical(i, num_classes=n_tags) for i in y]
#print(y)
test_pred = model.predict(X, verbose=1)
pred_labels = pred2label(test_pred)
test_labels = pred2label(y)
print(test_labels,pred_labels)
#print(pred_labels)

Where
do
you
want
to
go
?
56
Torstrasse
[('Where', 'ADV', ''), ('do', 'AUX', ''), ('you', 'PRON', ''), ('want', 'VERB', ''), ('to', 'PART', ''), ('go', 'VERB', ''), ('?', 'PUNCT', ''), ('56', 'NUM', ''), ('Torstrasse', 'PROPN', '')]
[[75, 84, 29, 58, 37, 27, 32, 92, 92]]
1/1 [==============================] - 0s 22ms/step
[['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']] [['', '', '', '', '', '', '', '', 'Destination', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']]


In [21]:
print(word2idx)

{'mind': 1, 'have': 2, "'": 3, ',': 4, 'Kid': 5, 'Charlottenstrasse': 6, 'show': 7, 'route': 8, 'railway': 9, 'destination': 10, 'St.': 11, 'pizzeria': 12, 'book': 13, 'Do': 14, 'take': 15, '3100': 16, 'me': 17, 'Jalgaon': 18, 'as': 19, '!': 20, 'nt': 21, 'school': 22, 'Main': 23, 'work': 24, 'children': 25, 'stop': 26, 'go': 27, 'quickly': 28, 'you': 29, 'headings': 30, 'bother': 31, '?': 32, "n't": 33, 'alexanderplatz': 34, 'navigate': 35, 'kids': 36, 'to': 37, 'one': 38, '76': 39, 'give': 40, 'directions': 41, 'gas': 42, "'s": 43, 'police': 44, 'get': 45, 'got': 46, '45': 47, 'kid': 48, 'the': 49, 'my': 50, 'up': 51, 'place': 52, 'ignore': 53, 'starbucks': 54, 'closest': 55, 'schutzenstrasse': 56, 'store': 57, 'want': 58, 'North': 59, 'coffee': 60, 'next': 61, 'drive': 62, 'What': 63, 'Never': 64, 'nearest': 65, 'for': 66, 'XYYZlet': 67, 'Unknown': 92, 'home': 69, 'parents': 70, '23': 71, 'station': 72, 'cafe': 73, 'let': 74, 'Where': 75, 'Friedrichstrasse': 76, 'address': 77, 'shop